## Note: Distance calculation between spatial points (locations)

Objective is to demonstrate how to calcualate distance between two locations in meters.

### Setup
Let's first install required packages to runtime environment

In [1]:
%%capture
%pip install duckdb
%pip install geopandas
%pip install tabulate

### Explanations
Import

In [2]:
import duckdb
import pandas as pd



Connect to DuckDB

In [3]:
con = duckdb.connect()

Install spatial library.

In [4]:
%%time
query = """
INSTALL spatial; LOAD spatial;
"""
results = con.sql(query)
print(type(results))

<class 'NoneType'>
CPU times: total: 1.11 s
Wall time: 279 ms


Let's check the first location in ESPS_4326 and ESPS_3857

In [5]:
#POI: (43.03714745364597 , -76.13331032552375)
NNQ_LON=-76.13331032552375
NNQ_LAT= 43.03714745364597
NNQ_RADIUS = 110

query = f"""
SELECT ST_Point({NNQ_LAT}, {NNQ_LON}) AS P1_EPSG_4326,
       ST_Transform(ST_Point({NNQ_LAT}, {NNQ_LON}), 'EPSG:4326', 'EPSG:3857') AS P1_EPSG_3857;
"""
results = con.sql(query)
print(type(results))
print(results)

<class '_duckdb.DuckDBPyRelation'>
┌──────────────────────────────────────────────┬─────────────────────────────────────────────┐
│                 P1_EPSG_4326                 │                P1_EPSG_3857                 │
│                   geometry                   │                  geometry                   │
├──────────────────────────────────────────────┼─────────────────────────────────────────────┤
│ POINT (43.03714745364597 -76.13331032552375) │ POINT (-8475121.33784358 5317627.778506153) │
└──────────────────────────────────────────────┴─────────────────────────────────────────────┘



Let's check the second location in ESPS_4326 and ESPS_3857

In [6]:
# v[1]=(43.03906983955765 , -76.13379936708955)
NNQ_LON2=-76.13379936708955
NNQ_LAT2= 43.03906983955765
query = f"""
SELECT ST_Point({NNQ_LAT2}, {NNQ_LON2}) AS P2_EPSG_4326,
       ST_Transform(ST_Point({NNQ_LAT2}, {NNQ_LON2}), 'EPSG:4326', 'EPSG:3857') AS P2_EPSG_3857;
"""
results = con.sql(query)
print(type(results))
print(results)

<class '_duckdb.DuckDBPyRelation'>
┌──────────────────────────────────────────────┬──────────────────────────────────────────────┐
│                 P2_EPSG_4326                 │                 P2_EPSG_3857                 │
│                   geometry                   │                   geometry                   │
├──────────────────────────────────────────────┼──────────────────────────────────────────────┤
│ POINT (43.03906983955765 -76.13379936708955) │ POINT (-8475175.777701663 5317920.566906542) │
└──────────────────────────────────────────────┴──────────────────────────────────────────────┘



The following code demonstrates these points in ESPG_4326 and EPSG_3857.

In [7]:
# con = duckdb.connect('food-delivery.db')
# P1
NNQ_LON=-76.13331032552375
NNQ_LAT= 43.03714745364597
NNQ_RADIUS = 110
# P2
NNQ_LON2=-76.13379936708955
NNQ_LAT2= 43.03906983955765
# Use Spatial functions
query = f"""
WITH points as (
SELECT 'P1' AS PID,
       ST_Point({NNQ_LAT}, {NNQ_LON}) AS EPSG_4326,
       ST_Transform(ST_Point({NNQ_LAT}, {NNQ_LON}), 'EPSG:4326', 'EPSG:3857') AS EPSG_3857
UNION ALL
SELECT 'P2' AS PID,
       ST_Point({NNQ_LAT2}, {NNQ_LON2}) AS EPSG_4326,
       ST_Transform(ST_Point({NNQ_LAT2}, {NNQ_LON2}), 'EPSG:4326', 'EPSG:3857') AS EPSG_3857
)
SELECT PID,
       ST_AsText(EPSG_4326) AS EPSG_4326,
       ST_AsText(EPSG_3857) AS EPSG_3857,
FROM points
"""
results = con.sql(query).fetchall()

# Define DataFrame
df = pd.DataFrame(results,columns=["PID", "EPSG_4326", "EPSG_3857"])
df

,PID,EPSG_4326,EPSG_3857
0,P1,POINT (43.03714745364597 -76.13331032552375),POINT (-8475121.33784358 5317627.778506153)
1,P2,POINT (43.03906983955765 -76.13379936708955),POINT (-8475175.777701663 5317920.566906542)


## Calculate distance

It continued to use the same two points.
<br>Distance between two points in ESPS_4326 is in degrees. (reference: L-11)
<br>Distance between two points in ESPS_3858 is in meters. (reference: L-12)
<br>It also added correction factor based on [ST_Distance](https://postgis.net/docs/ST_Distance.html). (reference: L-13)
<br>

In [8]:
import math

correction = math.cos(NNQ_LAT)
query = f"""
with points as (
SELECT  ST_Point({NNQ_LAT}, {NNQ_LON}) AS P1_EPSG_4326
      , ST_Point({NNQ_LAT2}, {NNQ_LON2}) AS P2_EPSG_4326
      , ST_Transform(ST_Point({NNQ_LAT}, {NNQ_LON2}), 'EPSG:4326', 'EPSG:3857') AS P1_EPSG_3857
      , ST_Transform(ST_Point({NNQ_LAT2}, {NNQ_LON2}), 'EPSG:4326', 'EPSG:3857') AS P2_EPSG_3857
)
SELECT ST_Distance(P1_EPSG_4326,P2_EPSG_4326) as dist_P1P2_EPSG_4326,  -- L-11
       ST_Distance(P1_EPSG_3857,P2_EPSG_3857) as dist_P1P2_EPSG_3857,  -- L-12
       (ST_Distance(P1_EPSG_3857,P2_EPSG_3857)*{correction}) as dist_P1P2_EPSG_3857_corrected -- L-13
FROM points
"""
results = con.sql(query)
print(type(results))
print(results)

<class '_duckdb.DuckDBPyRelation'>
┌──────────────────────┬─────────────────────┬───────────────────────────────┐
│ dist_P1P2_EPSG_4326  │ dist_P1P2_EPSG_3857 │ dist_P1P2_EPSG_3857_corrected │
│        double        │       double        │            double             │
├──────────────────────┼─────────────────────┼───────────────────────────────┤
│ 0.001983615196173787 │  292.78840038832277 │            171.46319504547327 │
└──────────────────────┴─────────────────────┴───────────────────────────────┘



dist_P1P2_EPSG_4326 is in degrees.
<br>dist_P1P2_EPSG_3857 is in meters.
<br>dist_P1P2_EPSG_3857_corrected is in meters.
<br>

## Summary

This notebook has demonstrate the distance calculation between two locations by using **spatial** library.

## References
1. https://postgis.net/
1. https://postgis.net/docs/ST_Transform.html
1. https://postgis.net/docs/ST_Distance.html
1. https://postgis.net/docs/ST_Point.html
1. https://postgis.net/docs/ST_AsText.html